In [34]:
!pip uninstall -y pytorch-lightning
!pip install lightning==2.6.0 pytorch-forecasting==1.5.0 torchmetrics==1.8.2


Found existing installation: pytorch-lightning 2.6.0
Uninstalling pytorch-lightning-2.6.0:
  Successfully uninstalled pytorch-lightning-2.6.0
  Using cached pytorch_lightning-2.6.0-py3-none-any.whl.metadata (21 kB)
Using cached pytorch_lightning-2.6.0-py3-none-any.whl (849 kB)


In [35]:
import torch
import pandas as pd

import lightning.pytorch as pl
from lightning.pytorch import Trainer

from pytorch_forecasting import TimeSeriesDataSet, Baseline
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss


In [17]:
df = pd.read_csv("final_tft_dataset.csv", parse_dates=["date"])

# Make sure dtype is correct
df["time_idx"] = df["time_idx"].astype("int64")
df["symbol"] = df["symbol"].astype("category")

df.head()


,date,close,volume,macd_line,macd_diff,macd_signal,rsi,vix_prct_returns,daily_prct_change,ebitda,ev_ebitda,daily_return,log_return,target,time_idx,symbol
0,2016-04-01,24.96,113856948,0.584986,0.073510,0.511476,73.167919,-6.093190,0.930044,16464.0,38.940719,-0.060932,0.009257,0.000812,0,AAPL
1,2016-04-04,25.21,157112212,0.618475,0.085599,0.532876,75.319795,7.786260,1.001603,16464.0,38.940719,0.077863,0.009966,0.003644,3,AAPL
2,2016-04-05,24.92,111266284,0.614530,0.065323,0.549207,68.460985,9.206799,-1.150337,16464.0,38.940719,0.092068,-0.011570,0.012852,4,AAPL
3,2016-04-06,25.18,111477272,0.625177,0.060776,0.564401,71.009868,-8.625162,1.043339,16464.0,38.940719,-0.086252,0.010379,0.014263,5,AAPL
4,2016-04-07,24.63,132677620,0.582520,0.014495,0.568025,59.969013,14.691270,-2.184273,16464.0,38.940719,0.146913,-0.022085,0.000786,6,AAPL


In [44]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_lightning import Trainer


In [45]:
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["symbol", "time_idx"]).reset_index(drop=True)

print(df.head())
print(df.columns)

        date  close     volume  macd_line  macd_diff  macd_signal        rsi  \
0 2016-04-01  24.96  113856948   0.584986   0.073510     0.511476  73.167919   
1 2016-04-04  25.21  157112212   0.618475   0.085599     0.532876  75.319795   
2 2016-04-05  24.92  111266284   0.614530   0.065323     0.549207  68.460985   
3 2016-04-06  25.18  111477272   0.625177   0.060776     0.564401  71.009868   
4 2016-04-07  24.63  132677620   0.582520   0.014495     0.568025  59.969013   

   vix_prct_returns  daily_prct_change   ebitda  ev_ebitda  daily_return  \
0         -6.093190           0.930044  16464.0  38.940719     -0.060932   
1          7.786260           1.001603  16464.0  38.940719      0.077863   
2          9.206799          -1.150337  16464.0  38.940719      0.092068   
3         -8.625162           1.043339  16464.0  38.940719     -0.086252   
4         14.691270          -2.184273  16464.0  38.940719      0.146913   

   log_return    target  time_idx symbol  
0    0.009257  0.00

In [46]:
target = "target"
group_ids = ["symbol"]
time_idx = "time_idx"

# ✅ Features known in the future
known_reals = ["time_idx"]

# ✅ Observed only in the past
unknown_reals = [
    "close",
    "volume",
    "macd_line",
    "macd_diff",
    "macd_signal",
    "rsi",
    "vix_prct_returns",
    "daily_prct_change",
    "daily_return",
    "log_return"
]

# ✅ Static (fundamentals)
static_reals = [
    "ebitda",
    "ev_ebitda"
]


In [47]:
max_encoder_length = 60     # lookback window (2–3 months)
max_prediction_length = 10 # forecast horizon (2 weeks)

training_cutoff = df[time_idx].max() - max_prediction_length


In [50]:
try:
    training = TimeSeriesDataSet(
        df[df[time_idx] <= training_cutoff],
        time_idx=time_idx,
        target=target,
        group_ids=group_ids,

        static_reals=static_reals,
        time_varying_known_reals=known_reals,
        time_varying_unknown_reals=unknown_reals,

        max_encoder_length=max_encoder_length,
        max_prediction_length=max_prediction_length,

        target_normalizer=None,

        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,

        allow_missing_timesteps=True
    )
    print("✅ TimeSeriesDataSet created successfully!")
    
except AssertionError as e:
    print("❌ AssertionError:", e)
    print("Known Reals:", known_reals)
    print("Unknown Reals:", unknown_reals)
    print("Static Reals:", static_reals)


✅ TimeSeriesDataSet created successfully!


In [51]:
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

train_loader = DataLoader(training, batch_size=64, shuffle=True)
val_loader = DataLoader(validation, batch_size=64, shuffle=False)


In [52]:
from torch.utils.data import DataLoader

validation = TimeSeriesDataSet.from_dataset(
    training,
    df,
    predict=True,
    stop_randomization=True
)


In [53]:
train_loader = DataLoader(
    training,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    persistent_workers=True
)

val_loader = DataLoader(
    validation,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    persistent_workers=True
)


In [54]:
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=7,    # quantiles
    loss=QuantileLoss(),
    reduce_on_plateau_patience=4,
)


c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [18]:
max_encoder_length = 60
max_prediction_length = 1

# Use last ~10% of time for validation
max_time_idx = df["time_idx"].max()
val_cutoff = max_time_idx -  int(0.1 * (max_time_idx - df["time_idx"].min()))

print("Max time_idx:", max_time_idx)
print("Validation starts at time_idx >", val_cutoff)


Max time_idx: 3498
Validation starts at time_idx > 3149


In [19]:
# all features except target/time_idx/symbol/date
feature_cols = [
    "close",
    "volume",
    "macd_line",
    "macd_diff",
    "macd_signal",
    "rsi",
    "vix_prct_returns",
    "daily_prct_change",
    "ebitda",
    "ev_ebitda",
    "daily_return",
    "log_return",
]

training = df[df["time_idx"] <= val_cutoff].copy()
validation = df[df["time_idx"] > val_cutoff].copy()

print("Train shape:", training.shape)
print("Val shape  :", validation.shape)


Train shape: (2172, 16)
Val shape  : (238, 16)


In [20]:
training_dataset = TimeSeriesDataSet(
    training,
    time_idx="time_idx",
    target="target",
    group_ids=["symbol"],
    max_encoder_length=max_encoder_length,
    min_encoder_length=max_encoder_length,     # fix encoder length
    max_prediction_length=max_prediction_length,
    min_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=feature_cols + ["target"],
    static_categoricals=["symbol"],
    target_normalizer=None,   # or leave None if target already scaled / log-return
    allow_missing_timesteps=True,
)

validation_dataset = training_dataset.from_dataset(
    training_dataset, validation, stop_randomization=True
)
batch_size = 64

train_dataloader = training_dataset.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0
)
val_dataloader = validation_dataset.to_dataloader(
    train=False, batch_size=batch_size, num_workers=0
)



In [21]:
baseline_predictions = Baseline().predict(val_dataloader)
print("Baseline prediction shape:", baseline_predictions.shape)


c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Baseline prediction shape: torch.Size([111, 1])


c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


In [40]:


tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=1e-3,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    loss=QuantileLoss([0.1, 0.5, 0.9]),  # ✅ explicit quantiles
    output_size=3,                      # ✅ MUST match number of quantiles
)


print(f"Model has {tft.size()/1e3:.1f}k parameters")


Model has 84.6k parameters


In [41]:
from lightning.pytorch.core import LightningModule

print(type(tft))
print(isinstance(tft, LightningModule))


<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>
True


In [42]:
trainer = Trainer(
    max_epochs=20,
    accelerator="auto",
    gradient_clip_val=0.1,
)

trainer.fit(
    model=tft,   # ✅ MUST be named
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

   | Name                               | Type                            | Params | Mode  | FLOPs
--------------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train | 0    
1  | logging_metrics                    | ModuleList                      | 0      | train | 0    
2  | input_embeddings                   | MultiEmbedding                  | 1      | train | 0    
3  | prescalers                         | ModuleDict                      | 448    | train | 0    
4  | static_variable_selection          | VariableSelectionNetwork        | 96     | train | 0    
5  | encoder_variabl

c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
c:\Users\john2\anaconda3\envs\611\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:317: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 19: 100%|██████████| 18/18 [00:02<00:00,  7.83it/s, v_num=4, train_loss_step=0.0086, val_loss=0.013, train_loss_epoch=0.0107]   

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 18/18 [00:02<00:00,  7.68it/s, v_num=4, train_loss_step=0.0086, val_loss=0.013, train_loss_epoch=0.0107]


In [31]:
# Point prediction using median quantile (0.5)
y_pred = tft.predict(val_dataloader).detach().cpu().numpy().flatten()

# Ground truth
y_true = x["decoder_target"].detach().cpu().numpy().flatten()


<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>
False


In [33]:
lightning.pytorch


NameError: name 'lightning' is not defined